In [2]:
import sys

sys.path.append("../")

import numpy as np
import pandas as pd
from dotenv import load_dotenv

load_dotenv()
import os
import sqlite3

DB_PATH = os.getenv("DB_PATH")

conn = sqlite3.connect(DB_PATH)

In [33]:
# Individuals Regions
df_ind_regions = pd.read_sql_query("SELECT * FROM individuals_regions", conn)

# Years
df_ind = pd.read_sql_query("SELECT * FROM individuals_main_information", conn)
df_ind_year = df_ind[["individual_wikidata_id", "birthyear"]].drop_duplicates()
df_ind_year = df_ind_year.dropna()
df_ind_year = df_ind_year[df_ind_year['birthyear']<=1850]


temporal_resolution = 10
df_ind_year["decade"] = df_ind_year["birthyear"].apply(lambda x: round(x / temporal_resolution) * temporal_resolution)

df_catalogs_id = pd.read_sql_query("SELECT * FROM individual_identifiers", conn)

df_catalogs = pd.read_sql_query("SELECT * FROM identifiers", conn)
df_catalogs = df_catalogs[['identifiers_wikidata_id', 'country_name']].dropna()

df_cat = pd.merge(df_catalogs_id, df_catalogs, on = 'identifiers_wikidata_id')
df_cat = df_cat[['individual_wikidata_id','individual_name', 'country_name']].drop_duplicates()
df_cat = df_cat.groupby(['individual_name', 'individual_wikidata_id'])['country_name'].count().rename('score').reset_index()
df_cat = df_cat.sort_values('score', ascending=False).reset_index(drop=True)

df_final = pd.merge(df_ind_year, df_cat, on = 'individual_wikidata_id')
df_final = pd.merge(df_final, df_ind_regions, on = ['individual_wikidata_id', 'individual_name'])

,individual_wikidata_id,birthyear,decade,individual_name,score
0,Q6121397,1841.0,1840,Santos Domínguez y Benguria,1
1,Q55696859,1824.0,1820,Albert Lemoine,15
2,Q861742,1694.0,1690,Edmond Plumier,4
3,Q1611181,1824.0,1820,Hermann Gebhardt,3
4,Q13129165,1805.0,1800,Hugh Hughes,3
...,...,...,...,...,...
195125,Q41260928,1830.0,1830,Albert Usteri,3
195126,Q4228920,1850.0,1850,Bruno Eugen Julius Kolbe,2
195127,Q60442324,1818.0,1820,Ferdinand Raisp,2
195128,Q55914754,1675.0,1680,Georg Heinrich Paritius,6


In [20]:
#list(set(df_catalogs.country_name))